In [1]:
from common import *

In [2]:
data = load_json('3.json', True)
data_flat = sorted([flatten_json(
        d, flatten_keys=['gadget', 'motionInfo', 'gatherGadget'], flatten_xyz=True, flatten_lists=False) for d in data],
        key=lambda x: (len(x), x.get('gadgetId', 0)))
df = pd.DataFrame(data_flat)
df1 = df.loc[:,df.nunique(dropna=False)!=1].sort_values(['itemId', 'groupId', 'entityId', 'configId', 'gadgetId'])

def make_gatherable_templates(df: pd.DataFrame) -> list:
    def make_template(df, name):
        common_values = df.loc[:, df.nunique(dropna=False) == 1]
        varying_values = df.loc[:, df.nunique(dropna=False) > 1]
        output = {'TemplateName': name}
        # for k,v in common_values.iloc[0].dropna().iteritems():
        #     output[k] = v
        # output['zChildren'] = varying_values.dropna().to_dict('records')
        for k,v in common_values.iloc[0].iteritems():
            output[k] = v
        output['zChildren'] = varying_values.to_dict('records')
        return output

    gatherable_counts = df['itemId'].value_counts()
    one_ofs = gatherable_counts[gatherable_counts == 1].index
    multiple_ofs = gatherable_counts[gatherable_counts > 1].index
    outputs = []
    for gatherable_id in multiple_ofs:
        name = handbook_items.get(int(gatherable_id), gatherable_id)
        outputs.append(make_template(df[df['itemId'] == gatherable_id], name))
    outputs.append(make_template(df[df['itemId'].isin(one_ofs)], 'one_ofs'))
    outputs.append(make_template(df[df['itemId'].isnull()], 'NaNs'))
    return outputs

t = make_gatherable_templates(df)

In [3]:
def encode_json(obj, starting_indent=0, maxline=180, spacing='') -> str:
    def wrap(contents: list, indent: int):
        total_len = sum(len(s) for s in contents)
        if total_len < maxline:
            return ','.join(contents)
        else:
            pre: str = '\n' + '\t' * indent
            pre2: str = '\n' + '\t' * (indent + 1)
            return pre2 + (','+pre2).join(contents) + pre
    
    def not_nan(v):
        if isinstance(v, (np.number, float)):
            return v == v
        return True

    def encode(obj, indent=0):
        if isinstance(obj, list) or isinstance(obj, tuple):
            contents = [encode(o, indent+1) for o in obj]
            return '[' + wrap(contents, indent) + ']'
        if isinstance(obj, dict):
            contents = [f'{encode(k)}:{spacing}{encode(v, indent+1)}' for k,v in obj.items() if not_nan(v)]
            return '{' + wrap(contents, indent) + '}'
        if obj != obj:
            return '>>>>>>>>>>>>>>>>>>>>>AAAAAAAA A NAN AAAAAAAAAAAAAAAAA<<<<<<<<<<<<<<<<<<<<<<<' + str(type(obj))
        if isinstance(obj, bool):
            return 'true' if obj else 'false'
        if isinstance(obj, str):
            if obj == 'zChildren':
                return '"Children"'
            return f'"{obj}"'
        if isinstance(obj, int):
            return str(obj)
        if isinstance(obj, float):
            out = str(obj)
            if out.endswith('.0'):
                return out[:-2]
            return out
        return str(obj)
    return encode(obj, starting_indent)

ts = encode_json(t, -2)
with open('3.pandas.json', 'w', encoding='utf-8', newline='\n') as f:
    f.write(ts)

In [4]:
def load_group(filename):
    with open(filename, 'r') as file:
        data = file.read()
        # _, _, data = data.partition('\n')
    try:
        lua = lupa.LuaRuntime(unpack_returned_tuples=True)
        lua.execute(data)
        g = lua.globals()
        d = dict(g)
        gadgets = g['gadgets']
        return table_to_dict(gadgets)
    except lupa.LuaError as e:
        print(filename, e)
path = '/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/3/'  # scene3_group133102124.lua
filenames = [s for s in os.listdir(path) if s.endswith('.lua')]

In [5]:
group_filenames = [s for s in filenames if s.startswith('scene3_group')]
groups = {s[12:-4]:load_group(path+s) for s in group_filenames}
with open('3.groups.lua.json', 'w', encoding='utf-8', newline='\n') as file:
    file.write(encode_json(groups))

/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/3/scene3_group131061695.lua [string "<python>"]:94: attempt to index a nil value (local 'L4_1')
stack traceback:
	[string "<python>"]:94: in main chunk
/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/3/scene3_group131061696.lua [string "<python>"]:65: attempt to index a nil value (local 'L3_1')
stack traceback:
	[string "<python>"]:65: in main chunk
/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/3/scene3_group132203486.lua [string "<python>"]:49: attempt to index a nil value (local 'L3_1')
stack traceback:
	[string "<python>"]:49: in main chunk
/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/3/scene3_group133001004.lua [string "<python>"]:127: attempt to index a nil value (local 'L4_1')
stack traceback:
	[string "<python>"]:127: in main chunk
/home/luke/code/GenshinImpact/Grasscutter/resources/Scripts/Scene/3/scene3_group133001006.lua [string "<python>"]:107: attempt

In [6]:
block_filenames = [s for s in filenames if s.startswith('scene3_block')]
blocks = {s[12:-4]:load_group(path+s) for s in block_filenames}
with open('3.blocks.lua.json', 'w', encoding='utf-8', newline='\n') as file:
    file.write(encode_json(blocks))

In [7]:
GatherExcelConfigData = pd.read_json('resources/ExcelBinOutput/GatherExcelConfigData.json')
GatherExcelConfigData['itemId'] = GatherExcelConfigData['itemId'].map(handbook_items.get)

In [30]:
keys = textmaps['EN'].keys()
reverse_mappings = {}
for key in keys:
    st = tuple(textmaps[l].get(key, '') for l in ('EN', 'JP', 'CHS', 'CHT', 'RU'))
    reverse_mappings[st] = reverse_mappings.get(st, []) + [key]

reverse_mappings_sorted = sorted(reverse_mappings.items(), key=lambda x: len(x[1]), reverse=True)
for st, hashes in reverse_mappings_sorted[:2500]:
    # print(f'{st}: {len(hashes)} copies: {", ".join(hashes)}')
    print(f'{len(hashes)} copies of \t{st}')

AttributeError: 'list' object has no attribute 'items'

In [35]:
for st, hashes in reverse_mappings_sorted[:2500]:
    # print(f'{st}: {len(hashes)} copies: {", ".join(hashes)}')
    print(f'{len(hashes)} copies of \t{st}')

354543 copies of 	('', '', '', '', '')
1830 copies of 	('Paimon', 'パイモン', '派蒙', '派蒙', 'Паймон')
1029 copies of 	('Quest Item', '任務アイテム', '任务道具', '任務道具', 'Задание')
697 copies of 	('...', '……', '……', '……', '...')
653 copies of 	('Unlocks at Friendship Lv. 4', '好感度Lv.4後に解放', '好感等级达到4后开启', '好感等級達到4後開放', 'Уровень дружбы 4')
647 copies of 	('2-Hit DMG|{param2:F1P}', '2段ダメージ|{param2:F1P}', '二段伤害|{param2:F1P}', '二段傷害|{param2:F1P}', 'Урон атаки 2|{param2:F1P}')
644 copies of 	('???', '？？？', '？？？', '？？？', '???')
608 copies of 	('1-Hit DMG|{param1:F1P}', '1段ダメージ|{param1:F1P}', '一段伤害|{param1:F1P}', '一段傷害|{param1:F1P}', 'Урон атаки 1|{param1:F1P}')
456 copies of 	('3-Hit DMG|{param3:F1P}', '3段ダメージ|{param3:F1P}', '三段伤害|{param3:F1P}', '三段傷害|{param3:F1P}', 'Урон атаки 3|{param3:F1P}')
444 copies of 	('Skill DMG|{param1:P}', 'スキルダメージ|{param1:P}', '技能伤害|{param1:P}', '技能傷害|{param1:P}', 'Урон навыка|{param1:P}')
407 copies of 	('Furnishing Blueprint', '調度品設計図', '摆设图纸', '擺設圖紙', 'Чертёж декора')
397 copies

In [7]:
textmaps = load_textmaps()
OptionExcelConfigData = pd.read_json(GC_PATH + 'resources/ExcelBinOutput/OptionExcelConfigData.json')
OptionExcelConfigData['contentTextMapHash'] = OptionExcelConfigData['contentTextMapHash'].map(lambda x: textmaps['EN'].get(str(x)))
OptionExcelConfigData.to_excel('OptionExcelConfigDataNames.xlsx', index=False)

In [8]:
gadget_ids = {}
for entity in data_flat:
    id = entity.get('gadgetId', 0)
    gatherable = entity.get('itemId', 0)
    gadget_ids[id] = gadget_ids.get(id, []) + [gatherable]

for gadget_id, item_ids in gadget_ids.items():
    if len(set(item_ids)) > 1:
        num_of_each = {}
        for id in item_ids:
            num_of_each[id] = num_of_each.get(id, 0) + 1
        print(gadget_id, [f'{handbook_items.get(id, id)}: {v}' for id,v in num_of_each.items()])

70590025 [0, 'Scarlet Quartz']
70540003 [0, 'Apple']
70520017 ['Fowl', 0]
70540021 [0, 'Sunsettia']
